# **Projeto Lógica de Programação II**

## Enunciado

[X] Ler um arquivo formato JSON.

[X] Realizar um mapeamento

[X] Realizar um filtro

[X] Realizar uma redução

[X] Permitir que os dados podem ser lidos individualmente, atualizados e deletados (manter JSON atualizado);

[ ] Garantir que todas as operações tenham validações (try/except, raise);

[X] Obter algum dado estatístico simples, como média de um grupo (exemplo, a média de idade dos professores que dão aula de exatas, caso o conjunto de dados seja do tema professor);

[X] Criar uma função para obter uma lista de tuplas, com máximo (ou minimo) valor de algum atributo numérico, com a primeira posição contendo o nome do elemento e a segunda o valor máximo (o motivo de ser uma lista é porque pode ter mais de um elemento com valor máximo);

[X] Esta função deve ser um parametro opcional, que diz qual das estatisticas você deseja obter (máximo ou minimo);

[X] Obter algum dado estatístico simples, como média, máximo ou minimo (exemplo, a média de idade dos professores que dão aulas de exatas, caso o conjunto de dados seja do tema professor);

[X] Salvar dados estatísticos em um CSV.

## Resolução

#### Importando libs e criando funções para o .json

In [1]:
import json
import csv

# definindo função para leitura de arquivo .json (precisa carregar o .json aqui no colab)
def open_json_file(filepath: str = 'petshop.json') -> list:
    try:
        with open(filepath, "r") as f:
            # lendo e desserializando o conteúdo do arquivo
            return json.load(f)
    # caso não encontre o arquivo, para o programa
    except FileNotFoundError as e:
        raise e

# definindo função para salvar dados
def save_json_file(data: list, filepath: str = 'petshop.json') -> bool:
    try:
        with open(filepath, 'w') as f:
            f.write(json.dumps(data))
            return True
    except Exception:
        return False

# função para salvar csv
def save_csv(data: list, attribute: str, max_or_min: str, filename: str = 'petshop.csv'):
    try:
        with open(filename, mode='w', newline='') as f:
            writer = csv.writer(f)

            writer.writerow(['Média:', data[0]])

            writer.writerow(['Nome', attribute, max_or_min])
            writer.writerows(data[1])
    except Exception as e:
        raise e

#### Funções CRUD

In [2]:
# função para validar opções feita pelo professor em aula
def valida_opcoes(valor: str, opcoes: list) -> bool:
    return valor in opcoes

# função para obter opções feita pelo professor em aula
def obter_opcoes(opcoes, msg='Opções'):
    msg = f"{msg} ({' | '.join([f'{key} - {values}' for key, values in opcoes.items()])}):"

    while True:
        valor = input(msg).upper()

        if valida_opcoes(valor, opcoes.keys()):
            break

        msg = f'Entrada Inválida! As opções validas são {", ".join(opcoes.keys())} \n' + msg

    return valor

# função para obter valor feita pelo professor em aula
def obter_valor(msg='', func=float):

    while True:
        valor = input(msg)
        try:
            return list(map(func, [valor]))[0]
        except ValueError:
            msg = f'Entrada Inválida! {msg}'

# funções para receber o input de opções
def input_name() -> str:
    return input("Enter the pet's name: ").title()

def input_age() -> int:
    return obter_valor("Enter the pet's age: ", func=int)

def input_species() -> str:
    return input("Enter the pet's species: ").capitalize()

def input_breed() -> str:
    return input("Enter the pet's breed: ").title()

def input_weight() -> float:
    return obter_valor("Enter the pet's weight: ", func=float)

def input_fur() -> str:
    return input("Enter the pet's fur color: ").title()

def input_attribute(options) -> str:
    return obter_opcoes(options, 'Choose an attribute: ').lower()

def entering_func(data) -> str:
    return print(f'Entering filter on {data}...')

# formata um pet
def formatting_pet_data(pet: dict) -> str:
    return '\n'.join([f'{key.title()}: {value}' for key, value in pet.items()])

# formata todos os pets - USANDO MAP
def formatting_all_pets(pets: dict):
    return '\n\n'.join(list(map(formatting_pet_data, pets)))

# procurando o nome do animal dentro dos dados
def find_pet(data: dict, name: str) -> list:
  return [pet for pet in data if pet['name'] == name]

# retorna todos os pets
def show_all(pets: dict)-> bool:
    print(formatting_all_pets(pets))
    return True

def insert_new(pets: dict) -> bool:
    pets.append({
        'name': input_name(),
        'age': input_age(),
        'species': input_species(),
        'breed': input_breed(),
        'weight': input_weight(),
        'fur_color': input_fur()
    })

    return True

def delete(pets: dict) -> bool:
    deleted = find_pet(pets, input_name())

    if len(deleted) == 0:
        print(f'{deleted} does not exist in our database!')
        return False

    deleted = deleted[0]

    msg = f'Are you sure you want to delete the following pet: [{formatting_pet_data(deleted)}]?'

    if obter_opcoes({'Y': 'Yes', 'N': 'No'}, msg) == 'Y':
        pets.remove(deleted)
        return True
    else:
        return False

def exec_alteration_update(data: dict) -> None:
    opc = {
        'N': 'name',
        'A': 'age',
        'S': 'species',
        'B': 'breed',
        'W': 'weight',
        'F': 'fur_color',
        'E': 'end'
    }

    while True:
        match obter_opcoes(opc, 'Escolha o campo'):
            case 'E':
                break
            case 'N':
                data['name'] = input_name()
            case 'A':
                data['age'] = input_age()
            case 'S':
                data['species'] = input_species()
            case 'B':
                data['breed'] = input_breed()
            case 'W':
                data['weight'] = input_weight()
            case 'F':
                data['fur_color'] = input_fur()

def update(pets: dict) -> bool:
    updated = find_pet(pets, input_name())

    if len(updated) == 0:
        print(f'{updated} does not exist in our database!')
        return False

    updated = updated[0]
    msg = f'Are you sure you want to update the following pet: [{formatting_pet_data(updated)}]'

    if obter_opcoes({'Y': 'Yes', 'N': 'No'}, msg) == 'Y':
        exec_alteration_update(updated)
        return True
    else:
        return False

#### Funções Estatisticas

In [3]:
from functools import reduce

# função para filtrar
def filter_data(data):
    option = input('Choose what attribute you want to filter: name, age, species, breed, weight, fur_color: ').lower()
    filter_value = input(f'Enter the value for {option}: ').strip().capitalize()  # Capturar o valor a ser filtrado

    if option in ('name', 'species', 'breed', 'fur_color'):
        filter_value = filter_value.capitalize()  # Garantir que as strings sejam capitalizadas para comparação
        return list(filter(lambda pet: str(pet.get(option)).strip().capitalize() == filter_value, data))
    elif option == 'age':
        filter_value = int(filter_value)  # Converter a entrada para inteiro
        return list(filter(lambda pet: int(pet.get(option)) == filter_value, data))
    elif option == 'weight':
        filter_value = float(filter_value)  # Converter a entrada para float
        return list(filter(lambda pet: float(pet.get(option)) == filter_value, data))
    else:
        return []

# função para calcular a soma de idades dos animais
def sum_age(data: list[dict], attribute: str) -> float:
    return reduce(lambda x, pet: x + pet[attribute], data, 0)

# função para calcular a média de idade dos animais
def average_age(data: dict, reduce_function) -> float:
    return (reduce_function / len(data))

def return_statistics_avarage(data, att):
    return average_age(data, sum_age(data, att))

# função para retornar o registro maximo de um atributo
def max_attribute_value(data: dict, attribute: str) -> list:
    max_value = max(data, key=lambda x: x[attribute])[attribute]
    return [(pet['name'], pet[attribute]) for pet in data if pet[attribute] == max_value]

# função para retornar o registro minimo de um atributo
def min_attribute_value(data: dict, attribute: str) -> list:
    min_value = min(data, key=lambda x: x[attribute])[attribute]
    return [(pet['name'], pet[attribute]) for pet in data if pet[attribute] == min_value]

opc_max_min = {
    'MAX': 'Máximo', #salvando dados
    'MIN': 'Mínimo'
}
opc_att = {'AGE': "Pet's age", 'WEIGHT': "Pet's weight"}

def input_max_min() -> str:
    return obter_opcoes(opc_max_min, 'Choose an input: ')

def max_min(opc_max_min: str = input_max_min , att: str = input_attribute, data = dict) -> list:
    if opc_max_min == 'MAX':
        return max_attribute_value(data, att)
    elif opc_max_min == 'MIN':
        return min_attribute_value(data, att)
    else:
        return []

#### Funções para rodar ao final do programa

In [4]:
# função para executar as estatisticas
def run_statistics(data, attribute, max_or_min):
    try:
        avarage = return_statistics_avarage(data, attribute)
        max_min_statistic = max_min(max_or_min, attribute, data)
        return [avarage, max_min_statistic]
    except Exception as e:
        raise e

# função para sair do programa e retornar as estatisticas no csv
def func_exit(data):
    try:
        attribute = input_attribute({'AGE': "Pet's age", 'WEIGHT': "Pet's weight"})
        max_or_min = input_max_min()
        statistics_list = run_statistics(data, attribute, max_or_min)
        if obter_opcoes({'S': 'Sim', 'N': 'Não'}, 'Deseja Sair') == 'S':
            return save_csv(statistics_list, attribute, max_or_min)
    except Exception as e:
        raise e

#### Executando o código

In [ ]:
opc_pet = {
    'NAME': "Pet's name",
    'AGE': "Pet's age" ,
    'SPC': "Pet's species",
    'BREED': "Pet's breed",
    'WEIGHT': "Pet's weight",
    'FUR': "Pet's fur color",
}

opc = {
    'I': 'Insert', #OK
    'U': 'Update', #OK
    'D': 'Delete', #OK
    'F': 'Find', #OK
    'SA': 'Show All', #OK
    'FI': 'Filter', #OK
    'EX': 'Exit' #OK
}

opc_func = {
    'I': insert_new,
    'D': delete,
    'U': update,
    'SA': show_all,
    'FI': entering_func,
    'F': lambda data: print(formatting_all_pets(find_pet(data, input_name()))),
    'EX': func_exit
}

s = True
while s == True:
    data = open_json_file()
    option = obter_opcoes(opc, 'Choose and action: ')
    opc_func[option](data)
    save_json_file(data)

    if option == 'FI':
        print(filter_data(data))
    elif option == 'EX':
        s = False

Choose and action:  (I - Insert | U - Update | D - Delete | F - Find | SA - Show All | FI - Filter | EX - Exit):i
Enter the pet's name: Pericles
Enter the pet's age: 1
Enter the pet's species: Dog
Enter the pet's breed: Pug
Enter the pet's weight: 7
Enter the pet's fur color: White
Choose and action:  (I - Insert | U - Update | D - Delete | F - Find | SA - Show All | FI - Filter | EX - Exit):f
Enter the pet's name: Pericles
Name: Pericles
Age: 1
Species: Dog
Breed: Pug
Weight: 7.0
Fur_Color: White
Choose and action:  (I - Insert | U - Update | D - Delete | F - Find | SA - Show All | FI - Filter | EX - Exit):u
Enter the pet's name: pericles
Are you sure you want to update the following pet: [Name: Pericles
Age: 1
Species: Dog
Breed: Pug
Weight: 7.0
Fur_Color: White] (Y - Yes | N - No):y
Escolha o campo (N - name | A - age | S - species | B - breed | W - weight | F - fur_color | E - end):w
Enter the pet's weight: 8
Escolha o campo (N - name | A - age | S - species | B - breed | W - weigh

## Arquivo Json do chatgpt:
  

In [ ]:
{
  "pets": [
    {"name": "Buddy", "age": 2, "species": "Dog", "breed": "Golden Retriever", "weight": 25, "fur_color": "Golden"},
    {"name": "Whiskers", "age": 1, "species": "Cat", "breed": "Siamese", "weight": 8, "fur_color": "Cream"},
    {"name": "Rocky", "age": 3, "species": "Dog", "breed": "German Shepherd", "weight": 35, "fur_color": "Black/Tan"},
    {"name": "Fluffy", "age": 4, "species": "Cat", "breed": "Persian", "weight": 10, "fur_color": "White"},
    {"name": "Charlie", "age": 1, "species": "Dog", "breed": "Beagle", "weight": 18, "fur_color": "Tri-color"},
    {"name": "Mittens", "age": 2, "species": "Cat", "breed": "Maine Coon", "weight": 15, "fur_color": "Tabby"},
    {"name": "Max", "age": 5, "species": "Dog", "breed": "Labrador Retriever", "weight": 30, "fur_color": "Chocolate"},
    {"name": "Oreo", "age": 1, "species": "Cat", "breed": "Domestic Shorthair", "weight": 7, "fur_color": "Black/White"},
    {"name": "Coco", "age": 3, "species": "Dog", "breed": "Poodle", "weight": 15, "fur_color": "Apricot"},
    {"name": "Smokey", "age": 2, "species": "Cat", "breed": "Ragdoll", "weight": 12, "fur_color": "Blue Bicolor"},
    {"name": "Bailey", "age": 4, "species": "Dog", "breed": "Boxer", "weight": 22, "fur_color": "Fawn"},
    {"name": "Shadow", "age": 1, "species": "Cat", "breed": "Siberian", "weight": 14, "fur_color": "Gray"},
    {"name": "Daisy", "age": 3, "species": "Dog", "breed": "Dachshund", "weight": 12, "fur_color": "Red"},
    {"name": "Milo", "age": 2, "species": "Cat", "breed": "Bengal", "weight": 18, "fur_color": "Spotted Rosetted"},
    {"name": "Chloe", "age": 5, "species": "Dog", "breed": "Shih Tzu", "weight": 10, "fur_color": "White/Gold"},
    {"name": "Leo", "age": 1, "species": "Cat", "breed": "Scottish Fold", "weight": 9, "fur_color": "Blue Tabby"},
    {"name": "Riley", "age": 3, "species": "Dog", "breed": "Husky", "weight": 28, "fur_color": "Gray/White"},
    {"name": "Luna", "age": 2, "species": "Cat", "breed": "Sphynx", "weight": 6, "fur_color": "Pink"},
    {"name": "Teddy", "age": 4, "species": "Dog", "breed": "Cavalier King Charles Spaniel", "weight": 13, "fur_color": "Blenheim"},
    {"name": "Oliver", "age": 1, "species": "Cat", "breed": "British Shorthair", "weight": 11, "fur_color": "Blue"}
    {"name": "Pericles", "age": 1, "species": "Dog", "breed": "Pug", "weight": 7, "fur_color": "White"}
    {"name": "Madalena", "age": 5, "species": "Dog", "breed": "Pug", "weight": 8, "fur_color": "Black"}
    {"name": "Boris", "age": 10, "species": "Dog", "breed": "Pug", "weight": 12, "fur_color": "Beige/Black"}
  ]
}

{'pets': [{'name': 'Buddy',
   'age': 2,
   'species': 'Dog',
   'breed': 'Golden Retriever',
   'weight': 25,
   'fur_color': 'Golden'},
  {'name': 'Whiskers',
   'age': 1,
   'species': 'Cat',
   'breed': 'Siamese',
   'weight': 8,
   'fur_color': 'Cream'},
  {'name': 'Rocky',
   'age': 3,
   'species': 'Dog',
   'breed': 'German Shepherd',
   'weight': 35,
   'fur_color': 'Black/Tan'},
  {'name': 'Fluffy',
   'age': 4,
   'species': 'Cat',
   'breed': 'Persian',
   'weight': 10,
   'fur_color': 'White'},
  {'name': 'Charlie',
   'age': 1,
   'species': 'Dog',
   'breed': 'Beagle',
   'weight': 18,
   'fur_color': 'Tri-color'},
  {'name': 'Mittens',
   'age': 2,
   'species': 'Cat',
   'breed': 'Maine Coon',
   'weight': 15,
   'fur_color': 'Tabby'},
  {'name': 'Max',
   'age': 5,
   'species': 'Dog',
   'breed': 'Labrador Retriever',
   'weight': 30,
   'fur_color': 'Chocolate'},
  {'name': 'Oreo',
   'age': 1,
   'species': 'Cat',
   'breed': 'Domestic Shorthair',
   'weight': 7,
 

## Solução gpt

Pessoal, inseri esse código aqui gerado pelo gpt. Creio que alguns pedaços possam ser adaptados para nossa solução

In [ ]:
import json
import functools
import csv
import os

def ler_json(arquivo):
    try:
        with open(arquivo, 'r') as f:
            dados = json.load(f)
        return dados
    except FileNotFoundError:
        print(f"O arquivo {arquivo} não foi encontrado.")
        return None

def atualizar_json(arquivo, dados):
    try:
        with open(arquivo, 'w') as f:
            json.dump(dados, f)
    except Exception as e:
        print(f"Ocorreu um erro ao tentar atualizar o arquivo {arquivo}: {str(e)}")

def mapeamento(dados, funcao):
    try:
        return list(map(funcao, dados))
    except Exception as e:
        print(f"Ocorreu um erro ao tentar mapear os dados: {str(e)}")
        return None

def filtro(dados, funcao):
    try:
        return list(filter(funcao, dados))
    except Exception as e:
        print(f"Ocorreu um erro ao tentar filtrar os dados: {str(e)}")
        return None

def reducao(dados, funcao):
    try:
        return functools.reduce(funcao, dados)
    except Exception as e:
        print(f"Ocorreu um erro ao tentar reduzir os dados: {str(e)}")
        return None

def estatistica(dados, atributo):
    try:
        total = sum(d[atributo] for d in dados)
        media = total / len(dados)
        maximo = max(d[atributo] for d in dados)
        minimo = min(d[atributo] for d in dados)
        return media, maximo, minimo
    except KeyError:
        print(f"O atributo {atributo} não existe nos dados.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao tentar calcular a estatística: {str(e)}")
        return None

def max_min(dados, atributo, estatistica='max'):
    try:
        if estatistica == 'max':
            valor = max(d[atributo] for d in dados)
        else:
            valor = min(d[atributo] for d in dados)
        return [(d['nome'], d[atributo]) for d in dados if d[atributo] == valor]
    except KeyError:
        print(f"O atributo {atributo} não existe nos dados.")
        return None
    except Exception as e:
        print(f"Ocorreu um erro ao tentar obter o {estatistica} valor: {str(e)}")
        return None

def salvar_csv(arquivo, dados):
    try:
        with open(arquivo, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(dados)
    except Exception as e:
        print(f"Ocorreu um erro ao tentar salvar os dados em um arquivo CSV: {str(e)}")
